Tweets With Emoji
Tweets dataset of 43 types of tweets with 20k tweets per emoji 🥰

About Dataset
The data was obtained through the utilization of snscrape. The query used for retrieval was based on individual emojis. Relevant data was identified, and subsequently assessed for the presence of emojis as well as the sentence's adherence to English language conventions. The language detection analysis was conducted using pycld3, which was inspired by the paper "The WiLI benchmark dataset for written language identification." Each csv file consists of 20,000 distinct data entries. The file name is created based on emoji package (emoji.EMOJI_DATA) in Python.

It should be noted that given the possible occurrence of small errors associated with pycld3, along with the potential for multiple emojis per data entry, there may exist instances of non-English tweets or duplicated tweets across different CSV files.

In [21]:
import os, re, unicodedata, csv
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
DATASET_DIR = "/kaggle/input/tweets-with-emoji"
OUTPUT_DIR = "/kaggle/working"
TEST_SIZE = 0.1
RNG_SEED = 42
random.seed(RNG_SEED)

In [23]:
FILENAME_TO_EMOJI = {
    "backhand_index_pointing_right.csv": "👉",
    "check_mark.csv": "✔️",
    "check_mark_button.csv": "✅",
    "clown_face.csv": "🤡",
    "cooking.csv": "🍳",
    "egg.csv": "🥚",
    "enraged_face.csv": "😡",
    "eyes.csv": "👀",
    "face_holding_back_tears.csv": "🥹",
    "face_savoring_food.csv": "😋",
    "face_with_steam_from_nose.csv": "😤",
    "face_with_tears_of_joy.csv": "😂",
    "fearful_face.csv": "😨",
    "fire.csv": "🔥",
    "folded_hands.csv": "🙏",
    "ghost.csv": "👻",
    "grinning_face_with_sweat.csv": "😅",
    "hatching_chick.csv": "🐣",
    "hot_face.csv": "🥵",
    "loudly_crying_face.csv": "😭",
    "melting_face.csv": "🫠",
    "middle_finger.csv": "🖕",
    "party_popper.csv": "🎉",
    "partying_face.csv": "🥳",
    "pile_of_poo.csv": "💩",
    "rabbit.csv": "🐇",
    "rabbit_face.csv": "🐰",
    "red_heart.csv": "❤️",
    "rolling_on_the_floor_laughing.csv": "🤣",
    "saluting_face.csv": "🫡",
    "skull.csv": "💀",
    "smiling_face.csv": "☺️",
    "smiling_face_with_halo.csv": "😇",
    "smiling_face_with_heart-eyes.csv": "😍",
    "smiling_face_with_hearts.csv": "🥰",
    "smiling_face_with_sunglasses.csv": "😎",
    "smiling_face_with_tear.csv": "🥲",
    "sparkles.csv": "✨",
    "sun.csv": "☀️",
    "thinking_face.csv": "🤔",
    "thumbs_up.csv": "👍",
    "white_heart.csv": "🤍",
    "winking_face.csv": "😉",
}

In [25]:
dfs = []
for fname, emo in FILENAME_TO_EMOJI.items():
    fpath = os.path.join(DATASET_DIR, fname)
    # newline='' để csv.reader xử lý newline đúng chuẩn CSV (kể cả trong quotes)
    with open(fpath, "r", encoding="utf-8", errors="ignore", newline="") as f:
        reader = csv.reader(
            f,
            delimiter=",",
            quotechar='"',
            escapechar="\\",
            doublequote=True,
            quoting=csv.QUOTE_MINIMAL,
            skipinitialspace=False,
            strict=False,
        )
        rows = []
        for row in reader:
            if not row:
                continue
            # file kỳ vọng 1 cột; nếu “bẩn” có nhiều cột thì gộp lại bằng dấu phẩy
            text = row[0] if len(row) == 1 else ",".join(row)
            # bỏ BOM nếu dính ở đầu
            if text.startswith("\ufeff"):
                text = text[1:]
            rows.append(text)

    if not rows:
        continue
    df = pd.DataFrame({"text": rows})
    df["emoji"] = emo
    dfs.append(df)

full_df = pd.concat(dfs, ignore_index=True)
full_df = full_df.drop_duplicates(subset=["text", "emoji"]).reset_index(drop=True)

In [26]:
print(full_df.shape)

(852757, 2)


In [27]:
full_df.sample(5)

,text,emoji
476005,@RoseK24206400 @MariaGodinez25 @redbullracing ...,💩
536743,"@auntmaggiep Good Morning, Maggie! ☕️❤️🐇💐",❤️
829376,"To this wonderful pair of children of God, tha...",🤍
306984,Just me being cute👻 HAHAHHAHAH https://t.co/uh...,👻
677903,@AnnePParis @angie241325 Hope you’ve had a lov...,🥰


In [55]:
print("\nSố lớp (emoji):", full_df['emoji'].nunique())


Số lớp (emoji): 43


In [56]:
# Regex và hàm làm sạch
URL_RE        = re.compile(r"(https?://\S+|www\.\S+)", flags=re.IGNORECASE)
MENTION_RE    = re.compile(r"@\w+")
HASHTAG_RE    = re.compile(r"#\w+")
MULTISPACE_RE = re.compile(r"\s+")
EMOJI_RE      = re.compile(
    "["                     
    "\U0001F300-\U0001F5FF"
    "\U0001F600-\U0001F64F"
    "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FAFF"
    "\U00002700-\U000027BF"
    "\U00002600-\U000026FF"
    "\U00002B00-\U00002BFF"
    "\U00002300-\U000023FF"
    "]+",
    flags=re.UNICODE
)
EMOJI_EXTRA_RE = re.compile(r"[\u200d\ufe0e\ufe0f\U0001F3FB-\U0001F3FF]", flags=re.UNICODE)

def clean_text(s: str) -> str:
    t = unicodedata.normalize("NFKC", str(s))
    t = URL_RE.sub(" ", t)
    t = MENTION_RE.sub(" ", t)
    t = HASHTAG_RE.sub(" ", t)
    t = EMOJI_RE.sub(" ", t)
    t = EMOJI_EXTRA_RE.sub(" ", t)
    t = t.lower()
    t = MULTISPACE_RE.sub(" ", t).strip()
    return t

In [57]:
# Áp dụng làm sạch lên full_df
full_df["text"] = full_df["text"].map(clean_text)

# Bỏ các dòng rỗng sau khi làm sạch
full_df = full_df[full_df["text"] != ""].reset_index(drop=True)

In [58]:
print("Sau khi làm sạch:", full_df.shape)
full_df.sample(5)

Sau khi làm sạch: (835428, 2)


,text,emoji
432448,"the latest opportunities in dance study, perfo...",🎉
167210,tangina what if ot7 live after kcon,🥹
144629,maybe give the girls a snippet of one of the r...,👀
558868,humans finding neanderthals attractive and vic...,🤣
444326,have a nice weekend killa and congrats again i...,🎉


In [73]:
# Tách train/test
train_df, test_df = train_test_split(
    full_df, 
    test_size=TEST_SIZE, 
    random_state=RNG_SEED, 
    shuffle=True, 
    stratify=full_df['emoji']
)

In [75]:
print("Train set size: ", train_df.shape)
print("Test set size: ", test_df.shape)

Train set size:  (751885, 2)
Test set size:  (83543, 2)


In [76]:
# Lưu
train_path = os.path.join(OUTPUT_DIR, "train_data.csv")
test_path  = os.path.join(OUTPUT_DIR, "test_data.csv")
train_df.to_csv(train_path, index=False, encoding="utf-8")
test_df.to_csv(test_path, index=False, encoding="utf-8")